## let's start new here:

In [9]:
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [10]:
## let's start new here:


In [11]:
def get_movies_page(tmdb_url):

    needed_headers = {'User-Agent': "Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36"}
    response = requests.get(tmdb_url, headers = needed_headers )
    # Access the webpage using `requests`
    # Check if the request was successful
    if response.status_code != 200:
        raise Exception('Failed to load page {}'.format(tmdb_url))
    # Parse the `response' text using BeautifulSoup
    movies_doc = BeautifulSoup(response.text, 'html.parser')
    return movies_doc

#get_movies_page("https://www.themoviedb.org/movie?page=%d" %(i))


In [20]:
def get_movies_names(doc):

    movies_names_tags = doc.find_all('h2')[4:]  #Exclude the first 4 lines
    movies_names = []
    # Loop through the page get all the movie names from the page
    for h2 in movies_names_tags:
        movies_names.append(h2.a.text.strip())
    return movies_names

In [21]:
def get_movies_rating(doc):

    desc_selector = 'user_score_chart'
    movies_rating_tags = doc.find_all('div', {'class': desc_selector})
    movies_rating = []
    # Loop through the webpage to get the ratings of all the movies in the page
    for tag in movies_rating_tags:
        movies_rating.append(tag.attrs['data-percent'])
    return movies_rating


In [22]:
def get_movies_urls(doc):
    """
    Function to extract the movie links from HTML source code using BeautifulSoup. 
    """
    movies_urls = []
    base_url = 'https://www.themoviedb.org'
    movies_names_tags = doc.find_all('h2')[4:]  #starting from the 5th line
    # Loop through the webpage to get the URL of each movie
    for tag in movies_names_tags:
        movies_urls.append(base_url + tag.a['href'])
    return movies_urls

In [15]:
def get_detailed_movie_page(movies_url):
    """
    Function to read the HTML source code using BeautifulSoup.
    """

    movies_doc = get_movies_page(movies_url)
    return movies_doc
doc1 =get_detailed_movie_page(movies_url[1])

In [16]:
def get_movies_info(doc):
    """
    Function to get the movie informations - 
    release date, genre, runtime and director.
    """
    div1_tags = doc.find('div', class_ = 'facts')
    release_date = div1_tags.text.split()[1]
    genre = div1_tags.text.split()[3:-2]
    runtime = div1_tags.text.split()[-2:]
    
    div2_tags = doc.find_all('div', {'class':'scroller_wrap should_fade is_fading'})
    if div2_tags == []:
        director = 'None'
    else:
        director = div2_tags[0].text.strip().partition("\n")[0]

    div3_tags = doc.find('div', {'class':'grey_column'})
    a =re.sub(r'\n'," ",div3_tags.text)
    b = re.sub(' +', ' ', a).split(" ")
    budget = b[7]
    revenue = b[9]
    language = b[5]
    div2_tags = doc.find_all('div', {'class':'scroller_wrap should_fade is_fading'})
    
    div4_tags = doc.find('div', {'class':'overview',"dir":'auto'})
    
    return release_date, genre, runtime, director, budget, revenue, language


In [17]:
def get_all_movies_details(urls):

    genres = []
    release_dates = []
    runtimes = []
    directors = []
    budgets = []
    revenues = []
    languages = []
    
    # Loop through all the urls of the the movies 
    for url in urls:
        movie_doc = get_movies_page(url)
        # get_movies_info returns release_date, genre, runtime, director.
        release_date, genre, runtime, director, budget, revenue, language= get_movies_info(movie_doc)
        # Convert the genre list to string on ` `. 
        genres.append(" ".join(genre))
        release_dates.append(release_date)
        runtimes.append(" ".join(runtime))
        directors.append(director)
        budgets.append(budget)
        revenues.append(revenue)
        languages.append(language)
        
    return release_dates, genres, runtimes, directors, budgets, revenues, languages
        

In [18]:
def scrape_movies():
    """
    Function to download web page using `requests` and
    to extract the HTML source code using BeautifulSoup.
    """
    # Let's get the popular movies listing from the TMdb website
    page_count = 1 # Initializing the movie page count to 1
    # Define lists for all the movie attributes
    all_names = []
    all_ratings = []
    all_genres = []
    all_release_dates = []
    all_runtimes = []
    all_directors = []
    all_urls = []
    all_bud = []
    all_rev = []
    all_lan = []
    
    while page_count < 2: # Looping for 8 pages of the TMdb web page
        movies_url = "https://www.themoviedb.org/movie?page=%d" %(page_count)
        # Access the webpage using `requests`
        needed_headers = {'User-Agent': "Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36"}
        response = requests.get(movies_url, headers = needed_headers )
        if response.status_code != 200:
            raise Exception('Failed to load page {}'.format(movies_url))
        # Parse the `response' text using BeautifulSoup
        doc = BeautifulSoup(response.text, 'html.parser')
        
        urls = get_movies_urls(doc)
        release_date, genre, runtime, director, budgets, revenues, languages = get_all_movies_details(urls)
        
        # Append each movie attribute to respective lists
        all_names += get_movies_names(doc)
        all_ratings += get_movies_rating(doc)
        all_genres += genre
        all_release_dates += release_date
        all_runtimes += runtime
        all_directors += director
        all_urls += urls 
        all_bud += budgets
        all_rev += revenues
        all_lan += languages
        page_count += 1

        # Defining a dictionary to store the movie informations
    movies_dict = {
        'name': all_names,
        'rating': all_ratings,
        'genre': all_genres,
        'release_date': all_release_dates,
        'runtime': all_runtimes,
        'director': all_directors,
        'url': all_urls,
        'Buduget' : all_bud,
        'Revenue' : all_rev,
        "Original Language": all_lan
    }
    return pd.DataFrame(movies_dict)

In [19]:
scrape_movies()

,name,rating,genre,release_date,runtime,director,url,Buduget,Revenue,Original Language
0,Black Adam,73.0,"Action, Fantasy, Science Fiction",10/21/2022,2h 5m,Dwayne Johnson,https://www.themoviedb.org/movie/436270,"$200,000,000.00","$368,000,000.00",English
1,R.I.P.D. 2: Rise of the Damned,68.0,"Fantasy, Action, Comedy, Crime",11/15/2022,1h 42m,Jeffrey Donovan,https://www.themoviedb.org/movie/1013860,$130.00,"$78,324,220.00",English
2,Paradise City,58.0,"Crime, Action, Thriller",11/17/2022,1h 33m,John Travolta,https://www.themoviedb.org/movie/829799,"$20,000,000.00",-,English
3,Hex,38.0,"Action, Horror, Thriller",11/01/2022,1h 28m,Kayla Adams,https://www.themoviedb.org/movie/988233,-,-,English
4,Corrective Measures,49.0,"Science Fiction, Action",04/29/2022,1h 46m,Bruce Willis,https://www.themoviedb.org/movie/872177,-,-,English
5,Emily the Criminal,69.0,"Crime, Drama, Mystery, Thriller",08/12/2022,1h 37m,Aubrey Plaza,https://www.themoviedb.org/movie/862965,-,-,English
6,Disenchanted,74.0,"Comedy, Family,",11/18/2022,Fantasy 2h,Amy Adams,https://www.themoviedb.org/movie/338958,-,-,English
7,Lost Bullet 2,68.0,"Action, Drama, Thriller",11/10/2022,1h 38m,Alban Lenoir,https://www.themoviedb.org/movie/948276,Released,Language,2
8,Frank and Penelope,75.0,"Thriller, Horror, Crime",06/03/2022,1h 52m,Kevin Dillon,https://www.themoviedb.org/movie/899294,-,-,English
9,Margaux,68.0,"Horror, Science Fiction",09/09/2022,1h 45m,Madison Pettis,https://www.themoviedb.org/movie/846778,-,-,English


In [31]:
doc = get_movies_page("https://www.themoviedb.org/movie")
get_movies_names(doc)
get_movies_rating(doc)
movies_url = get_movies_urls(doc)
doc1 =get_detailed_movie_page(movies_url[1])

In [33]:
get_movies_info(doc1)


('11/15/2022',
 ['Fantasy,', 'Action,', 'Comedy,', 'Crime'],
 ['1h', '42m'],
 'Jeffrey Donovan',
 '$130.00',
 '$78,324,220.00',
 'English')

In [34]:
doc1

<!DOCTYPE html>

<html class="no-js" lang="en">
<head>
<title>R.I.P.D. 2: Rise of the Damned (2022) — The Movie Database (TMDB)</title>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="on" http-equiv="cleartype"/>
<meta charset="utf-8"/>
<meta content="Movies, TV Shows, Streaming, Reviews, API, Actors, Actresses, Photos, User Ratings, Synopsis, Trailers, Teasers, Credits, Cast" name="keywords"/>
<meta content="yes" name="mobile-web-app-capable"/>
<meta content="yes" name="apple-mobile-web-app-capable"/>
<meta content="width=device-width,initial-scale=1" name="viewport"/>
<meta content="When Sheriff Roy Pulsipher finds himself in the afterlife, he joins a special police force and returns to Earth to save humanity from the undead." name="description"/>
<meta content="/assets/2/v4/icons/mstile-144x144-30e7905a8315a080978ad6aeb71c69222b72c2f75d26dab1224173a96fecc962.png" name="msapplication-TileImage"/>
<meta content="#032541" name="msapplication-TileColor"/>
<meta con

In [37]:
div4_tags = doc1.find('div', {'class':'overview',"dir":'auto'})
div4_tags.text

'\nWhen Sheriff Roy Pulsipher finds himself in the afterlife, he joins a special police force and returns to Earth to save humanity from the undead.\n'